<a href="https://colab.research.google.com/github/nausheen53/SMAI/blob/master/5_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
import ssl
import sklearn
import numpy as np
import spacy
import re
import gensim
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
import textblob
from textblob import TextBlob, Word
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
class AuthorClassifier:
    labels_of_train=[]
    labels_of_test=[]
    final_list_train=[]
    final_list_test=[]
    
    def train(self,path):
        text_of_train = self.load(path)
        self.clean_data(text_of_train)
        
        
    def load(self,path):
        df = pd.read_csv(path,header = None)
        print(df)
        train=df.sample(frac=0.8,random_state=200)
        train = np.array(train)
        self.labels_of_train=[]
        for row in train:
            self.labels_of_train.append(row[2])
        text_of_train = []
        for row in train:
            text_of_train.append(row[1])
        return text_of_train
   
    def clean_data(self,text_of_train):
        import string
        from collections import Counter
        self.final_list_train=[]
        for sent in text_of_train:
            # print("1")
            sent = sent.lower()
            sent = sent.translate(str.maketrans('', '', string.punctuation))
            # print("sent1 ",sent)
            from nltk.tokenize import word_tokenize
            tokens = word_tokenize(sent)
            # remove all tokens that are not alphabetic
            words = [word for word in tokens if word.isalpha()]
            #remove stopwords
            from nltk.corpus import stopwords
            stop_words = set(stopwords.words('english'))
            words = [w for w in words if not w in stop_words]
            sent=" ".join(words)
            # print("sentttt ",sent)
            self.final_list_train.append(sent)

            
    def vectorise_test_train(self):
        from sklearn.feature_extraction.text import TfidfVectorizer
        from sklearn.preprocessing import normalize
        text = self.final_list_train
        vectorizer = TfidfVectorizer()
        vectorizer.fit(text)
        vector = vectorizer.fit_transform(text)
        vector = vector.todense()
        vector1 = vectorizer.transform(self.final_list_test)
        vector1 = vector1.todense()
        words = vectorizer.get_feature_names()
        vector_norm_train = normalize(vector)
        return vector,vector1
        

    
    def predict(self,path):
        text_of_test = self.load_test(path)
        self.clean_data_test(text_of_test)
        vector,vector1 =self.vectorise_test_train()
        prediction=self.classify(vector,vector1)
        return prediction
        
        
        
    def load_test(self,path):
        df = pd.read_csv(path,header = 0)
        test=df.sample(frac=0.8,random_state=200)
        test = np.array(test)
        self.labels_of_test=[]
        for row in test:
            self.labels_of_test.append(row[2])
        text_of_test = []
        for row in test:
            text_of_test.append(row[1])
        return text_of_test 
    
    def clean_data_test(self,text_of_test):
        import string
        from collections import Counter
        self.final_list_test=[]
        for sent in text_of_test:
            # print("1")
            sent = sent.lower()
            sent = sent.translate(str.maketrans('', '', string.punctuation))
            # print("sent1 ",sent)
            from nltk.tokenize import word_tokenize
            tokens = word_tokenize(sent)
            # remove all tokens that are not alphabetic
            words = [word for word in tokens if word.isalpha()]
            #remove stopwords
            from nltk.corpus import stopwords
            stop_words = set(stopwords.words('english'))
            words = [w for w in words if not w in stop_words]
            sent=" ".join(words)
            # print("sentttt ",sent)
            self.final_list_test.append(sent)
        
    
    def classify(self,vector,vector1):
        from sklearn.svm import LinearSVC
        clf = LinearSVC(random_state=0, tol=1e-5,max_iter=5000,C=1.0)
        clf.fit(vector,self.labels_of_train)
        # print(clf.predict(test))
        predicted_label=clf.predict(vector1)
        accu_score = accuracy_score(self.labels_of_test, predicted_label,normalize=True)
        print(accu_score) 
        f11 = f1_score(self.labels_of_test, predicted_label,average=None)
        re = recall_score(self.labels_of_test, predicted_label,average='weighted')
        cm = confusion_matrix(self.labels_of_test, predicted_label)
        pr_sc = precision_score(self.labels_of_test, predicted_label,average='weighted')
        clrp = classification_report(self.labels_of_test, predicted_label)
        print("**********LINEAR SVC WITH C= 1.0 ***********")
        print("accuracy score ",accu_score)
        print("********************")
        print("f1 score ",f11)
        print("********************")
        print("recall score ",re)
        print("********************")
        print("confusion matrix ",cm)
        print("********************")
        print("precision score ",pr_sc)
        print("********************")
        print("classification report ",clrp)
        return predicted_label

In [22]:
auth_classifier = AuthorClassifier()
auth_classifier.train('/content/drive/My Drive/Train.csv') # Path to the train.csv will be provided
predictions = auth_classifier.predict('/content/drive/My Drive/Train.csv') 


             0                                                  1       2
0          NaN                                               text  author
1          0.0  The sight of the military restored hope to tho...     MWS
2          1.0  Just as the building was wiped out by a German...     HPL
3          2.0                    Do you wonder how it will seem?     HPL
4          3.0  My revenge is of no moment to you; yet, while ...     MWS
...        ...                                                ...     ...
15659  15658.0                         Get bald, too, very young.     HPL
15660  15659.0  I performed the first part of my journey on ho...     MWS
15661  15660.0  He had a narrow head, bulging, watery blue eye...     HPL
15662  15661.0  There does not exist the man in England with a...     MWS
15663  15662.0  "Every man," he said, "dreams about something,...     MWS

[15664 rows x 3 columns]
0.958659217877095
**********LINEAR SVC WITH C= 1.0 ***********
accuracy score  0.95865

In [18]:
print(predictions)

['EAP' 'EAP' 'EAP' ... 'EAP' 'EAP' 'EAP']
